<a href="https://colab.research.google.com/github/EhsanHonarvar/Practices/blob/main/1_ACT_Sentiment140_Ver(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=2b8dbb2a0a52a4c7a9af027f7bcf9c9d0e0dbb46e7a2a8bbb9980248b58a5997
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop_words


In [ ]:
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from stop_words import get_stop_words
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re

In [ ]:
%matplotlib inline
import os
import random
import sys
from typing import List, Tuple, Dict, Set, Union
import re
import nltk
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
# full_data = pd.read_csv("/kaggle/input/sentiment140-dataset-1600000-tweets/training.1600000.processed.noemoticon.csv", names= ['Pos_Neg','id','date','query_string','user','Content'] , encoding = 'latin-1')

full_data = pd.read_csv("/content/drive/MyDrive/Thesis/training.1600000.processed.noemoticon.csv", names= ['Pos_Neg','id','date','query_string','user','Content'] , encoding = 'latin-1')
full_n = full_data.shape[0]

print(type(full_data))
print(full_n)
full_data.head(5)

<class 'pandas.core.frame.DataFrame'>
1600000


,Pos_Neg,id,date,query_string,user,Content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# # Content.Pos_Neg = Content.Pos_Neg.apply(lambda x: str(x).lower())
# full_data.Pos_Neg = full_data.Pos_Neg.map(lambda x: 1 if str(x)=='1' else 2)
# full_data.head()

In [ ]:
print(full_data.groupby(full_data.Pos_Neg).count())

             id    date  query_string    user  Content
Pos_Neg                                               
0        800000  800000        800000  800000   800000
4        800000  800000        800000  800000   800000


In [ ]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Pos_Neg       1600000 non-null  int64 
 1   id            1600000 non-null  int64 
 2   date          1600000 non-null  object
 3   query_string  1600000 non-null  object
 4   user          1600000 non-null  object
 5   Content       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
######## DATA CLEANING ########
stop_words = get_stop_words('english')

len(stop_words)

174

In [ ]:
# ps = PorterStemmer()
# corpus = []
# for i in full_data.Content:
#     new = re.sub('[^a-zA-z]', ' ',str(i))
#     new = new.lower()
#     new = nltk.word_tokenize(new)
#     new = [ps.stem(i) for i in new if i not in stop_words]
#     new = ' '.join(new)
#     corpus.append(new)

# full_data.text = corpus
# full_data.head()

In [ ]:
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''

    # convert to lowercase
    text = str(tweet).lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # remove links from text
    text = re.sub(r'http\S+', '', text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [ ]:
from tqdm import tqdm
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
cleantext=[]
for item in tqdm(full_data['Content']):
    words=tweet_to_words(item)
    cleantext+=[words]
full_data['cleantext']=cleantext

100%|██████████| 1600000/1600000 [39:21<00:00, 677.53it/s]


In [ ]:
full_data.head()

,Pos_Neg,id,date,query_string,user,Content,cleantext
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[switchfoot, http, twitpic, com, 2y1zl, awww, ..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, updat, facebook, text, might, cri, res..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, dive, mani, time, ball, manag, save..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, bodi, feel, itchi, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[nationwideclass, behav, mad, see]"


In [ ]:
################################################################
########################## Parameters ##########################
################################################################

# vocab_size = 20000  # Only consider the top 20k words
vocab_size = 20000  # Only consider the top 100k words
epochs =100
n_heads = 6   #Or 3
# maxlen = 200
maxlen = 20 # max_length was 64 or 32
num_class = 2
embed_dim = 128
################################################################

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, GlobalMaxPool1D,Dense,Flatten
from tensorflow.keras.models import Sequential, Model
from keras.layers import Conv1D,MaxPooling1D

In [ ]:
class TransformerBlock(Layer):
    def __init__(self, d_model, num_heads):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=d_model,)
        self.layernorm1 = LayerNormalization(epsilon=1e-6) #1e-6
        self.layernorm2 = LayerNormalization(epsilon=1e-6) #1e-6
        self.convnet=Sequential(
            [Conv1D(d_model,9,1,padding='same', activation='gelu'),MaxPooling1D(6,1,padding='same'),
             Conv1D(d_model,7,1,padding='same', activation='gelu'),MaxPooling1D(4,1,padding='same'),
             Conv1D(d_model,3,1,padding='same', activation='gelu'),MaxPooling1D(2,1,padding='same'),])
#adding layer normalization

    def call(self, inputs):
        cnn_output = self.convnet(inputs)
        ffn_output = self.layernorm1(cnn_output)
        attn_output = self.att(ffn_output,ffn_output)
        return self.layernorm2(inputs + attn_output)

In [ ]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, d_model):       # maxlen  <---------->  vocab_size
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=d_model)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=d_model)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
def tokenize_pad_sequences(text):
    # Text tokenization
    tokenizer = Tokenizer(num_words=vocab_size, lower=True, split=' ')
    tokenizer.fit_on_texts(text)
    # Transforms text to a sequence of integers
    X = tokenizer.texts_to_sequences(text)
    # Pad sequences to the same length
    X = pad_sequences(X, padding='post', maxlen=maxlen)
    # return sequences
    return X, tokenizer

In [ ]:
#create dataset train test val
from sklearn.model_selection import train_test_split
X, tokenizer = tokenize_pad_sequences(full_data['cleantext'])
y = pd.get_dummies(full_data['Pos_Neg'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
print('Train Set: ', X_train.shape, y_train.shape)
print('Validation Set: ', X_val.shape, y_val.shape)
print('Test Set: ', X_test.shape, y_test.shape)
print('X_train Sample: ',X_train[0:3])
print('y_train Sample: ',y_train[0:3])

Train Set:  (1280000, 28) (1280000, 2)
Validation Set:  (160000, 28) (160000, 2)
Test Set:  (160000, 28) (160000, 2)
X_train Sample:  [[   96     6     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [  269    37 10351    32  2543   669  1129   196   140   347  1983     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [  528 10909   954   759  1534  1227  7067  1667  1578    33  4024     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]
y_train Sample:           0  4
1556092  0  1
868905   0  1
218471   1  0


In [ ]:
batch_size = 128 # 64
#create model and training
inputs = Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, batch_size)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(batch_size, n_heads)

x = transformer_block(x)
# x = transformer_block(x) # Again
# x = GlobalAveragePooling1D()(x)
x = GlobalMaxPool1D()(x) # for testing
x = Dropout(.4)(x) #0.5
x = Dense(batch_size, activation='gelu')(x) #sigmoid
x = Dropout(.4)(x) #0.3
# outputs = Dense(3, activation="softmax")(x)
outputs = Dense(2, activation="sigmoid")(x) #softmax, num_class

In [ ]:
from tensorflow import keras
########
# epochs=70
########
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28)]              0         
                                                                 
 token_and_position_embeddin  (None, 28, 64)           1281792   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 28, 64)           177792    
 merBlock)                                                       
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0     

In [ ]:
# call back for learning rate, learning rate scheduler
# opt = keras.optimizers.Adam(learning_rate=0.00001) #0.0001

opt = tf.keras.optimizers.Adam(learning_rate=8e-6,epsilon=1e-08)

###################
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-2,
#     decay_steps=5,
#     decay_rate=0.9)
# opt = keras.optimizers.Adam(learning_rate=lr_schedule) # SGD
##################

model.compile(optimizer=opt , loss="binary_crossentropy", metrics=["accuracy"]) # "or mse" and "categorical_crossentropy"

history = model.fit(X_train,y_train, batch_size=64, epochs=epochs, validation_data=(X_val, y_val))  # batch_size = 256

results = model.evaluate(X_val, y_val, verbose=1)

Epoch 1/100
20000/20000 [==============================] - 212s 10ms/step - loss: 0.6076 - accuracy: 0.6524 - val_loss: 0.5088 - val_accuracy: 0.7513
Epoch 2/100
20000/20000 [==============================] - 176s 9ms/step - loss: 0.5019 - accuracy: 0.7599 - val_loss: 0.4822 - val_accuracy: 0.7689
Epoch 3/100
20000/20000 [==============================] - 175s 9ms/step - loss: 0.4815 - accuracy: 0.7735 - val_loss: 0.4724 - val_accuracy: 0.7747
Epoch 4/100
10141/20000 [==============>...............] - ETA: 1:23 - loss: 0.4724 - accuracy: 0.7790

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
# Print metrics
print('Accuracy  : {:.4f}'.format(accuracy))

In [ ]:
import matplotlib.pyplot as plt



fig, ax = plt.subplots(1,2, figsize=(10,4))
# first plot
ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Model Accuracy')
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('accuracy')
ax[0].legend(['train', 'validation'], loc='best')

# second plot
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Model Loss')
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('loss')
ax[1].legend(['train', 'validation'], loc='best')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
'''Function to plot confusion matrix for the passed model and the data'''

# sentiment_classes = ['Negative','Neutral', 'Positive']
sentiment_classes = ['Negative', 'Positive']

# use model to do the prediction
y_pred = model.predict(X_test)
# compute confusion matrix
cm = confusion_matrix(np.argmax(y_pred, axis=1),np.argmax(np.array(y_test),axis=1))

# plot confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(cm, cmap=plt.cm.Blues, annot=True, fmt='d',
            xticklabels=sentiment_classes,
            yticklabels=sentiment_classes)
plt.title('Confusion matrix', fontsize=16)
plt.xlabel('Actual label', fontsize=12)
plt.ylabel('Predicted label', fontsize=12)